# The shortest path between each pair of cities in Westeros
[Murwan Siddig](mailto:msiddig@clemson.edu)

In [2]:
#technical lines
using LightGraphs
using GraphPlot

In [5]:
# Define Westeros name vector
names = [
        "Castle Black",
        "Winterfell",
        "White Harbor",
        "Moat Cailin",
        "The Twins",
        "Riverrun",
        "The Trident",
        "Harrenhal",
        "The Eyrie",
        "Casterly Rock",
        "King's Landing",
        "Highgarden",
        "Summerhall",
        "Oldtown",
        "Horn Hill",
        "Nightsong",
        "Sunspear",
        "Storm's End"
        ]

# Define arcs
arcs = [
    1 2 680
    1 3 1105
    2 3 490
    2 4 590
    3 4 180
    4 5 530
    4 7 770
    4 11 1420
    5 6 495
    5 7 420
    6 7 255
    6 8 285
    6 11 750
    6 10 590
    7 8 195
    7 9 545
    8 11 380
    9 11 990
    10 11 1050
    10 12 730
    11 12 915
    11 13 380
    11 18 380
    12 14 390
    12 15 195
    12 13 640
    13 15 500
    13 18 350
    13 16 425
    14 15 320
    15 16 225
    16 17 1170
]
# Empty distance and adjacency matrices
distance = zeros(Int64,length(names),length(names))
adj = zeros(Int64,length(names),length(names))
# Populate them
for k in 1:size(arcs)[1]
    i=arcs[k,1]
    j=arcs[k,2]
    d=arcs[k,3]
    adj[i,j]=1
    adj[j,i]=1
    distance[i,j]=d
    distance[j,i]=d
end

GoT = DiGraph(adj);

{18, 64} directed simple Int64 graph

In [32]:
# Returns a vector with a matrix of the distances of the shortest paths between all pairs
# and a matrix of vectors showing the nodes of the shortest path for each pair
function allShortestPaths(g,arc_len,source=1)
    # Do bellman ford to get node potentials
    out=bellman_ford_shortest_paths(g,source,arc_len)
    # Modify arc length matrix
    arc_len2=copy(arc_len)
    for i in 1:size(arc_len2)[1]
        for j in 1:size(arc_len2)[1]
            if arc_len2[j,i]!=0
                arc_len2[j,i]+=-out.dists[i]+out.dists[j]
            end
        end 
    end
    # Define matrix for lenghts of shortest paths
    dists=copy(arc_len)
    # Define matrix for shortest paths
    paths=Array{Any,2}(undef, size(arc_len)[1], size(arc_len)[1])
    
    # Add shortest paths for starting location
    for j in 1:size(arc_len)[1]
        curr=out.parents[j]
        if curr==0 && source!=j
            path = []
        else
            path = [j]
        end
        path = [j]
        while curr!=0
            pushfirst!(path,curr)
            curr=out.parents[curr]
        end
        paths[source,j]=path
    end
    # Add distances for starting location    
    dists[source,:]=copy(out.dists)

    for i in 1:size(arc_len2)[1]
        if i==source
            continue
        end
        # Dijkstra
        djk=dijkstra_shortest_paths(g, i, arc_len2)
        
        # Distance of shortest path accouting for node potentials
        dists[i,:]=djk.dists.-out.dists[i]+out.dists
        # Add shortest paths to matrix
        for j in 1:size(arc_len)[1]
            curr=djk.parents[j]
            if curr==0 && i!=j
                path = []
            else
                path = [j]
            end
            while curr!=0
                pushfirst!(path,curr)
                curr=djk.parents[curr]
            end
            paths[i,j]=path
        end
    end
    # Get rid of max integers and add infs for unreachable nodes
    dists=float(dists)
    bigM = sum(abs.(arc_len))
    for i in 1:size(arc_len2)[1]
        for j in 1:size(arc_len2)[1]
            if abs(dists[j,i])>=bigM
                dists[j,i]=Inf
            end
        end 
    end
    return [dists,paths]
end

allShortestPaths (generic function with 2 methods)

In [33]:
# Pretty Print function
function printDistanceandPath(start,dest)
    s=findfirst(x->x == start, names)
    d=findfirst(x->x == dest, names)
    
    if s==0
        println("Invalid starting point")
        return
    end
    
    if d==0
        println("Invalid destination")
        return
    end
    println("The shortest path from ",names[s]," to ",names[d]," is:")
    for i in p[2][s,d]
        println("\t",names[i])
    end
    println("The path is ",p[1][s,d]," long.")
end

printDistanceandPath (generic function with 1 method)

In [37]:
p=allShortestPaths(GoT,distance,1);
printDistanceandPath("Sunspear","Castle Black")

The shortest path from Sunspear to Castle Black is:
	Sunspear
	Nightsong
	Summerhall
	King's Landing
	Harrenhal
	The Trident
	Moat Cailin
	Winterfell
	Castle Black
The path is 4590.0 long.
